In [2]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle


In [3]:
# Load CSV
df = pd.read_csv("exercise_angles.csv")

# Select the angle features
features = ["Shoulder_Angle", "Elbow_Angle", "Hip_Angle", "Knee_Angle", "Ankle_Angle"]

# Parameters
sequence_length = 120
X_seq = []
y_seq = []

# Sliding window
for i in range(0, len(df) - sequence_length + 1):
    window = df.iloc[i:i + sequence_length]
    if window['Label'].nunique() == 1:
        X_seq.append(window[features].values)
        y_seq.append(window['Label'].iloc[0])

# Convert to numpy
X_seq = np.array(X_seq)             
y_seq = np.array(y_seq)               
# Encode string labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_seq)

print("X_seq shape:", X_seq.shape)
print("y_encoded shape:", y_encoded.shape)
print("Classes:", label_encoder.classes_)


X_seq shape: (27038, 800, 5)
y_encoded shape: (27038,)
Classes: ['Jumping Jacks' 'Pull ups' 'Push Ups' 'Russian twists' 'Squats']


In [4]:
df = pd.read_csv("exercise_angles.csv")

# Assume the CSV has columns: 
# 'Shoulder_Angle', 'Elbow_Angle', 'Hip_Angle', 'Knee_Angle', 'Ankle_Angle', and 'Label'
features = ["Shoulder_Angle", "Elbow_Angle", "Hip_Angle", "Knee_Angle", "Ankle_Angle"]
label_col = "Label"

In [5]:
class ExerciseSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)  # Use long for CrossEntropyLoss

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [6]:
class ExerciseLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super(ExerciseLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # take last time step
        return out

In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=20):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Training Loss: {total_loss / len(train_loader):.4f}")

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                outputs = model(X_val)
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == y_val).sum().item()
                total += y_val.size(0)
        print(f"Validation Accuracy: {correct / total:.4f}")

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_encoded, test_size=0.2)

train_dataset = ExerciseSequenceDataset(X_train, y_train)
val_dataset = ExerciseSequenceDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Model setup
model = ExerciseLSTM(input_size=5, hidden_size=64, num_layers=2, num_classes=5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train
train_model(model, train_loader, val_loader, criterion, optimizer, device='cuda' if torch.cuda.is_available() else 'cpu')

Epoch 1, Training Loss: 0.1251
Validation Accuracy: 0.9976
Epoch 2, Training Loss: 0.0682
Validation Accuracy: 0.9795
Epoch 3, Training Loss: 0.0039
Validation Accuracy: 1.0000
Epoch 4, Training Loss: 0.0014
Validation Accuracy: 1.0000
Epoch 5, Training Loss: 0.0003
Validation Accuracy: 1.0000
Epoch 6, Training Loss: 0.0534
Validation Accuracy: 0.4859
Epoch 7, Training Loss: 0.3085
Validation Accuracy: 0.9541
Epoch 8, Training Loss: 0.0586
Validation Accuracy: 1.0000
Epoch 9, Training Loss: 0.0020
Validation Accuracy: 1.0000
Epoch 10, Training Loss: 0.0009
Validation Accuracy: 1.0000
Epoch 11, Training Loss: 0.0231
Validation Accuracy: 1.0000
Epoch 12, Training Loss: 0.0007
Validation Accuracy: 1.0000
Epoch 13, Training Loss: 0.0003
Validation Accuracy: 1.0000
Epoch 14, Training Loss: 0.0002
Validation Accuracy: 1.0000
Epoch 15, Training Loss: 0.0005
Validation Accuracy: 1.0000
Epoch 16, Training Loss: 0.0001
Validation Accuracy: 1.0000
Epoch 17, Training Loss: 0.0001
Validation Accura

In [11]:
torch.save(model.state_dict(), "exercise_lstm_model.pth")
# print("Model weights saved to 'exercise_lstm_model.pth'")

# Save the model configuration so you can rebuild it later
model_config = {
    "input_size": 5,
    "hidden_size": 64,
    "num_layers": 2,
    "num_classes": len(label_encoder.classes_),
    "dropout": 0.3
}
with open("model_config.pkl", "wb") as f:
    pickle.dump(model_config, f)
print("Model config saved to 'model_config.pkl'")

# Save the label encoder to map predictions back to class names
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)
print("Label encoder saved to 'label_encoder.pkl'")

# Save the scaler if you used one (for angle normalization)
# with open("scaler.pkl", "wb") as f:
#     pickle.dump(scaler, f)
# print("Scaler saved to 'scaler.pkl'")

Model config saved to 'model_config.pkl'
Label encoder saved to 'label_encoder.pkl'
